## Vanilla OLS

Say we've got a dataset

In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
from yellowbrick.datasets import load_bikeshare

X, y = load_bikeshare()

Where we measure the number of DC-area bikes rented

In [ ]:
y.head()

Based on a number of features

In [ ]:
X.head()

And just shoving everything into a Logistic Regression seems to work... alright

In [ ]:
from statsmodels.api import OLS
import statsmodels.api as sm

X_const = sm.add_constant(X)
model = OLS(y, X_const).fit()
model.summary()

But at the same time, have good reason to believe that there's some colinearity/interaction at play with our features.

In [ ]:
from yellowbrick.features import Rank2D

visualizer = Rank2D(algorithm='pearson', size=(600, 500))
visualizer.fit_transform(X)
visualizer.poof();

## Interaction Terms 

From here, a good data scientist will take the time to do exploratory analysis and thoughtful feature engineering-- this is the "More Art than Science" adage you hear so often.

But we're trying to be home by 5, so how do we cram everything in and see what shakes out?

### Getting Values
Thankfully, the `PolynomialFeatures` object in `sklearn` has us mostly-covered.

It's originally used to generate sequences of `(b_i1 * x_i) + (b_i2 * x_i^2) + ...` for each feature in `X`, taking us from `n` features to `2^n` features (in the case of `PolynomialFeatures(degree=2)`, anyhow).

We're not interested in polynomials, per se, but if you squint, the same `itertools` magic™ that powers the backend of this can also be used to provide all pairwise feature combinations, with minimal rewriting. They provide this, ez pz, with the `interaction_only` flag.

And so we go from

In [ ]:
X.shape

To an expected $\frac{p * (p - 1)}{2}$ pairs, plus our original $p$ features, plus a bias term

In [ ]:
p = len(X.columns)
(p * (p-1)) / 2 + p + 1

Coolio

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

interaction = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_inter = interaction.fit_transform(X)
X_inter_const = sm.add_constant(X_inter)

X_inter_const.shape

Now we brazenly throw it into a new model, and... oh. A lot of features called `x`

In [ ]:
model = OLS(y, X_inter_const).fit()
model.summary()

### Feature Names?

The `interaction` object keeps track of the names of our feature combinations... sort of

In [ ]:
print(interaction.get_feature_names())

Assuming that a bevy of `xi xj`s aren't much more useful than the output of `model.summary()` we can do some hacky, Python nonsense to decode a bit.

For starters, we want to create a dictionary that maps `xi` to its corresponding feature name in our dataset.

We'll use the `itertools.count()` function, as it's basically `enumerate`, but plays better with generator expressions.

In [ ]:
from itertools import count

x_to_feature = dict(zip(('x{}'.format(i) for i in count()), X.columns))
x_to_feature

Next, you know it's Sound Data Science™ when we break out

In [ ]:
import re

Finally, this [little diddy](https://www.youtube.com/watch?v=9VBSH-2fMuw) goes through and makes the appropriate substitutions for `xi` to their respective feature names, then replaces spaces with underscores so `pandas` references isn't such a chore

In [ ]:
# necessary so `x11` gets a chance to swap before 
# before `x1` leaves us with "season1" where we wanted
# "windspeed"
feature_keys = list(x_to_feature.keys())[::-1]

features = []

for feature in interaction.get_feature_names():
    for key in feature_keys:
        feature_name = x_to_feature[key]
        feature = re.sub(key, feature_name, feature)
    feature = re.sub(' ', '_', feature)
    features.append(feature)

Then we'll slap these feature names onto our big ol' dataset

In [ ]:
import pandas as pd
import statsmodels.api as sm

X_all_inter = pd.DataFrame(X_inter, columns=features)
X_all_inter_const = sm.add_constant(X_all_inter)

And refit

In [ ]:
model = OLS(y, X_all_inter_const).fit()
model.summary()

### And that's Data Science!

In [ ]:
from IPython.display import Image
Image('images/we_did_it.jpg')